<a href="https://colab.research.google.com/github/NongNNew/Project_499/blob/main/Project_499_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [ ]:
!git clone https://github.com/NongNNew/Project_499.git

fatal: destination path 'Project_499' already exists and is not an empty directory.


In [ ]:
import pandas as pd
from glob import glob
import librosa
import librosa.display

In [ ]:
fruit_veget = ['กระชาย','กระท้อน','กระเทียม1','กระเทียม2','กระเพรา','กล้วยน้ำว้า','กล้วยหอม',
               'ข้าวโพด',
               'ตะขบไทย',
               'ทับทิม',
               'น้อยหน่า',
               'ฝรั่ง',
               'พุทรา',
               'มะปราง','มะพลับ','มะละกอ',
               'สับปะรด']

source = {'audio_time':[],
          'sampling_rate':[],
          'feature_extraction':[],
          'label':[],
          'description':[]}

In [ ]:
for types in fruit_veget:
    audio_files = glob('/content/Project_499/Record_audio/' + str(types) + '/*.wav')

    for audio in audio_files:
        y,sr = librosa.load(audio,duration=6,offset=0)
        source['description'].append(str(types))
        source['audio_time'].append(y)
        source['sampling_rate'].append(sr)
        source['feature_extraction'].append(librosa.feature.mfcc(y,sr,n_mfcc=13))
        source['label'].append(fruit_veget.index(types))

In [ ]:
import re

for i in range(len(source['description'])):
    source['description'][i] = re.findall('[\u0E00-\u0E7F]+',source['description'][i])[0]

In [ ]:
pd.DataFrame.from_dict(source)

,audio_time,sampling_rate,feature_extraction,label,description
0,"[-0.00021362305, -0.00024414062, -0.0002593994...",22050,"[[-662.2338, -663.28766, -665.21606, -668.8302...",0,กระชาย
1,"[1.5258789e-05, -4.5776367e-05, 0.0, 1.5258789...",22050,"[[-671.40265, -672.8409, -673.3839, -673.3374,...",0,กระชาย
2,"[-0.00010681152, -0.00010681152, -0.0001525878...",22050,"[[-622.78125, -623.396, -623.63983, -623.7701,...",0,กระชาย
3,"[-0.0002593994, -0.00022888184, -0.0002593994,...",22050,"[[-639.25616, -639.47345, -640.0064, -640.5056...",0,กระชาย
4,"[-0.004928589, -0.0049591064, -0.0048980713, -...",22050,"[[-634.622, -639.0867, -654.00525, -659.99115,...",0,กระชาย
...,...,...,...,...,...
335,"[-0.00010681152, -0.00019836426, -0.0002441406...",22050,"[[-688.85156, -688.6841, -685.51434, -680.2233...",16,สับปะรด
336,"[-0.00015258789, -0.00032043457, -0.0001678466...",22050,"[[-608.5522, -608.0134, -608.0346, -608.9745, ...",16,สับปะรด
337,"[-7.6293945e-05, -3.0517578e-05, -1.5258789e-0...",22050,"[[-620.5951, -621.67816, -622.1715, -620.5547,...",16,สับปะรด
338,"[-7.6293945e-05, -0.00016784668, -4.5776367e-0...",22050,"[[-636.96967, -637.10645, -638.38525, -638.639...",16,สับปะรด


In [ ]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(source['feature_extraction'])
y = np.array(source['label'])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=99)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 5, input_shape=(13, 259, 1), activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(64, 5, activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(np.unique(source['label'])), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train[:,:,:,None], y_train, epochs=100)

Epoch 1/100
8/8 [==============================] - 0s 11ms/step - loss: 104.8634 - accuracy: 0.0667
Epoch 2/100
8/8 [==============================] - 0s 10ms/step - loss: 3.8038 - accuracy: 0.0745
Epoch 3/100
8/8 [==============================] - 0s 10ms/step - loss: 2.8887 - accuracy: 0.0706
Epoch 4/100
8/8 [==============================] - 0s 9ms/step - loss: 2.8675 - accuracy: 0.0667
Epoch 5/100
8/8 [==============================] - 0s 8ms/step - loss: 2.8472 - accuracy: 0.0627
Epoch 6/100
8/8 [==============================] - 0s 8ms/step - loss: 2.8293 - accuracy: 0.0745
Epoch 7/100
8/8 [==============================] - 0s 7ms/step - loss: 2.8314 - accuracy: 0.0706
Epoch 8/100
8/8 [==============================] - 0s 8ms/step - loss: 2.8325 - accuracy: 0.0706
Epoch 9/100
8/8 [==============================] - 0s 7ms/step - loss: 2.8334 - accuracy: 0.0706
Epoch 10/100
8/8 [==============================] - 0s 8ms/step - loss: 2.8193 - accuracy: 0.0902
Epoch 11/100
8/8 [======

In [ ]:
model.summary()

Model: "sequential_101"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_202 (Conv2D)          (None, 9, 255, 64)        1664      
_________________________________________________________________
dropout_303 (Dropout)        (None, 9, 255, 64)        0         
_________________________________________________________________
max_pooling2d_202 (MaxPoolin (None, 4, 127, 64)        0         
_________________________________________________________________
conv2d_203 (Conv2D)          (None, 4, 127, 64)        102464    
_________________________________________________________________
dropout_304 (Dropout)        (None, 4, 127, 64)        0         
_________________________________________________________________
max_pooling2d_203 (MaxPoolin (None, 2, 63, 64)         0         
_________________________________________________________________
flatten_101 (Flatten)        (None, 8064)           

In [ ]:
model.evaluate(X_test[:,:,:,None],y_test)

3/3 [==============================] - 0s 6ms/step - loss: 0.0963 - accuracy: 0.9765


[0.09628892689943314, 0.9764705896377563]